In [1]:
%load_ext autotime

In [18]:
import nltk
import pickle
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tools.functions

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# nltk.download('stopwords')
# nltk.download('wordnet')

time: 5.07 ms


In [19]:
vectorizer_max_features = 1500

time: 241 µs


In [4]:
all_text = pd.read_csv('archive/sub_reddits.csv')
all_text['selftext'] = all_text['selftext'].astype(str)

time: 6.32 s


In [5]:
sanitized_posts_df = pd.DataFrame(tools.functions.sanitize_posts(all_text['selftext']), columns=['posts'])
sanitized_posts_df = sanitized_posts_df[sanitized_posts_df['posts'] != '']
sanitized_posts_df = sanitized_posts_df[sanitized_posts_df['posts'] != 'a']
sanitized_posts_df = sanitized_posts_df[sanitized_posts_df['posts'] != '_']
sanitized_posts_df = sanitized_posts_df[sanitized_posts_df['posts'].str.contains('[^\d^\s]')]

time: 2min 20s


In [20]:
# Workspace for additional text cleaning

time: 703 µs


In [ ]:
# Was this wrong?  I should only be running vectorizer.fit?

vectorizer = TfidfVectorizer(min_df=5, max_df=0.7, stop_words=stopwords.words('english'))

tfidfvectorized_posts = vectorizer.fit_transform(sanitized_posts_df['posts'])
.toarray()

In [21]:
count_vectorizer = CountVectorizer(min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
count_vectorized_posts = count_vectorizer.fit_transform(sanitized_posts_df['posts'])



time: 21.3 s


In [ ]:
vectorizer = TfidfVectorizer(min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
tfidf = vectorizer.fit(sanitized_posts_df['posts'])
tfidf_transformed = vectorizer.transform(sanitized_posts_df['posts'])

In [16]:
feature_names = vectorizer.get_feature_names()
tfidf_df = pd.DataFrame(tfidf_transformed.toarray(), columns=feature_names)

time: 22 s


In [17]:
tfidf_df.head()

,00,000,0000,00000,000000,000001,00001,0001,0001104659,0001193125,...,то,уоu,что,это,ісо,الاستثمار,في,كما,من,ﬁnancial
0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.12075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


time: 60.5 ms


In [22]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=2, n_iter=7, random_state=42)
svd.fit(count_vectorized_posts)


print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())
print(svd.singular_values_)

[0.13418152 0.11829433]
0.25247584468078477
[3170.21868301 2869.74127917]
time: 4.95 s


In [23]:
topic_word = pd.DataFrame(svd.components_.round(3), index=["component_1", "component_2"], columns=vectorizer.get_feature_names())
topic_word

,00,000,0000,00000,000000,000001,00001,0001,0001104659,0001193125,...,то,уоu,что,это,ісо,الاستثمار,في,كما,من,ﬁnancial
component_1,0.037,0.024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
component_2,-0.012,-0.005,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0


time: 63.8 ms


In [24]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

time: 478 µs


In [25]:
display_topics(svd, vectorizer.get_feature_names(), 5)


Topic  0
amp, http, com, gt, earnings

Topic  1
comment, wallstreetbets, 17, 15, spy
time: 37.7 ms


In [11]:
# for TF DTM
# lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
# lda_tf.fit(tfidfvectorized_posts)

# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf.fit(tfidf)

ValueError: Expected 2D array, got scalar array instead:
array=TfidfVectorizer(max_df=0.7, min_df=5,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...]).
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
with open('archive/lda.20.0.pkl', 'wb') as picklefile:
    pickle.dump(lda_tfidf, picklefile)

In [ ]:
with open('archive/lda.20.0.pkl', 'rb') as picklefile:
    lda_tfidf = pickle.load(picklefile)

In [ ]:
trans_lda = lda_tfidf.transform(tfidf)

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda_tfidf, tfidfvectorized_posts, TfidfVectorizer)